In [ ]:
# !pip install opencv-python
# !pip install opencv-contrib-python

In [1]:
import cv2
from cv2 import dnn_superres

file_name = 'Megan Is Missing'
ext = 'mp4'
scale = 4
input_video = f'./input/{file_name}.{ext}'
output_video = f'./output/videos/{file_name}_{scale}x_cubic.{ext}'

In [2]:
from tqdm import tqdm

# bicubic upsampling

vs = cv2.VideoCapture(input_video)
success, frame = vs.read()
input_width = frame.shape[1] # 640
input_height = frame.shape[0] # 352
frame_count = 0
total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
print(f'input res: {frame.shape}, total frames={total_frames}')

out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'),
                         24, (scale*input_width, scale*input_height))

with tqdm(total=100) as pbar:
    while success:
        upsampled_frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        out.write(upsampled_frame)

        frame_count += 1
        pbar.update(frame_count/total_frames * 100)
        pbar.set_description(f'frame no: {frame_count}')
        success, frame = vs.read()  
    pbar.close()

vs.release()
out.release()
print('done')

input res: (352, 640, 3), total frames=123062.0


frame no: 495: 100%|█████████▉| 99.75459524467341/100 [00:02<00:00, 76.55it/s]  /home/asutosh/miniconda3/envs/env_pytorch/lib/python3.8/site-packages/tqdm/std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
frame no: 123062: : 6153150.000000001it [10:41, 9596.81it/s]                   


done


In [3]:
import numpy as np
import os

sr = dnn_superres.DnnSuperResImpl_create()
fullModelName = 'FSRCNN_x4.pb'
path = f"./input/models/{fullModelName}"
modelName = fullModelName.split(os.path.sep)[-1].split("_")[0].lower()
modelScale = fullModelName.split("_x")[-1]
modelScale = int(modelScale[:modelScale.find(".")])

vs = cv2.VideoCapture(input_video)
success, frame = vs.read()
input_width = frame.shape[1] # 640
input_height = frame.shape[0] # 352
frame_count = 0
total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
print(f'input res: {frame.shape}, total frames={total_frames}')

sr.readModel(path)
sr.setModel(modelName, modelScale)

out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'),
                         24, (scale*input_width, scale*input_height))

with tqdm(total=100) as pbar:
    while success:
        upsampled_frame = sr.upsample(frame)
        out.write(upsampled_frame)

        frame_count += 1
        pbar.update(frame_count/total_frames * 100)
        pbar.set_description(f'frame no: {frame_count}')
        success, frame = vs.read()  
    pbar.close()

vs.release()
out.release()
print('done')

input res: (352, 640, 3), total frames=123062.0


frame no: 140:   8%|▊         | 8.020347467130389/100 [00:40<07:39,  4.99s/it]      


KeyboardInterrupt: 